In [3]:
import os
import numpy as np
import torch
import torchio as tio
import cv2
from PIL import Image
from torch.utils.data import Dataset
from src.configuration.config import IMAGE_HEIGHT, IMAGE_WIDTH

class CustomDatasetHW_3D(Dataset):
    def __init__(self, image_dir, mask_dir, datadict=datadict, output_size=(256, 256), output_depth=5):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.images = os.listdir(image_dir)
        self.series = os.listdir(mask_dir)
        self.datadict = datadict if datadict else {}
        self.reversed_dict = {v: k for k, v in self.datadict.items()}

        self.output_size = output_size  # (H, W)
        self.output_depth = (IMAGE_HEIGHT, IMAGE_WIDTH)   # Depth for resizing

        # Define 3D Transformations using TorchIO
        self.tio_transform = tio.Compose([
            tio.RandomAffine(scales=(0.9, 1.1), degrees=35),  # Random rotation & scaling
            tio.RandomFlip(axes=(0, 1), p=0.5),  # Flip along horizontal & vertical axes
            tio.ZNormalization(),  # Normalize
        ])

    def __len__(self):
        return len(self.series)

    def resize_volume(self, volume, new_depth):
        """Resize depth using linear interpolation."""
        d, h, w = volume.shape
        resized_volume = np.zeros((new_depth, h, w), dtype=volume.dtype)

        for i in range(new_depth):
            orig_idx = int(i * (d / new_depth))  # Interpolation
            resized_volume[i] = volume[orig_idx]

        return resized_volume

    def __getitem__(self, index):
        Maskvolume = []
        ImageVolume = []
        flag = 0

        for key in range(len(self.reversed_dict.keys())):
            category = self.reversed_dict[key]
            Maskcatgvolume = []
            Masks = os.path.join(self.mask_dir, os.listdir(self.mask_dir)[index], category)
            MasksList = sorted(os.listdir(Masks))
            
            for msk in MasksList:
                pngMask = np.array(Image.open(os.path.join(Masks, msk)))
                Maskcatgvolume.append(pngMask)
    
                if msk in self.images and flag == 0:
                    pngimage = np.array(Image.open(os.path.join(self.image_dir, msk)))
                    ImageVolume.append(pngimage)
            flag = 1

            Maskcatgvolume = np.stack(Maskcatgvolume, axis=0)
            Maskvolume.append(Maskcatgvolume)

        Maskvolume = np.stack(Maskvolume, axis=0)
        ImageVolume = np.stack(ImageVolume, axis=0)
        ImageVolume = np.expand_dims(ImageVolume, axis=0)

        # Convert multi-category masks to a single-class mask
        newMaskVolume = np.stack([np.argmax(Maskvolume[:, i, :, :], axis=0) for i in range(Maskvolume.shape[1])], axis=0)
        newMaskVolume = np.expand_dims(newMaskVolume, axis=0)

        # Apply 3D Transformations using TorchIO
        image_tensor = torch.tensor(ImageVolume, dtype=torch.float32)
        mask_tensor = torch.tensor(newMaskVolume, dtype=torch.int64)

        subject = tio.Subject(
            image=tio.ScalarImage(tensor=image_tensor),
            mask=tio.LabelMap(tensor=mask_tensor),
        )

        transformed_subject = self.tio_transform(subject)
        transformed_image = transformed_subject["image"].tensor.numpy()
        transformed_mask = transformed_subject["mask"].tensor.numpy()

        # Resize to the desired shape
        resized_images = np.array([cv2.resize(img, self.output_size, interpolation=cv2.INTER_LINEAR) for img in transformed_image[0]])
        resized_masks = np.array([cv2.resize(mask, self.output_size, interpolation=cv2.INTER_NEAREST) for mask in transformed_mask[0]])

        return torch.tensor(resized_images).unsqueeze(0), torch.tensor(resized_masks).unsqueeze(0)


In [2]:
import os
import cv2
import torch
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import numpy as np
from src.configuration.config import datadict, TrainingDir
from src.Dataset.dataset import CustomDatasetHW, CustomDatasetHWD, CustomDataset, CustomDatasetHW_new, CustomDatasetHW_validation
from src.utils.losses import BCEDiceLoss, DiceLoss, GeneralizedDiceLoss, WeightedCrossEntropyLoss, WeightedSmoothL1Loss
from src.configuration.config import IMAGE_HEIGHT, IMAGE_WIDTH
from src.utils.utils import custom_collate, custom_collate_Variable_HW
from src.Models.D_UNet import UNet3D

In [4]:
ImagesDir = os.path.join(TrainingDir, 'Images')
MasksDir = os.path.join(TrainingDir, 'Masks')
print(os.listdir(TrainingDir))
data = CustomDatasetHW_3D(ImagesDir, MasksDir)

['Images', 'Masks']


In [5]:
len(data)

8

In [6]:
inputs, targets = data[0]

In [7]:
inputs.shape

torch.Size([1, 9, 256, 256])

In [8]:
batch_size = 2
num_workers = 0
pin_memory = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(
    data,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=pin_memory,
    shuffle=True,
    collate_fn=custom_collate_Variable_HW,
)

In [9]:
def save_prediction(image, mask):
    gray_image = image
    binary_mask = mask
    
    gray_norm = gray_image / 255.0

    # Create an RGB image with grayscale as background
    overlay = np.stack([gray_norm, gray_norm, gray_norm], axis=-1)
    
    # Define lighter colors for each class (0-8)
    colors = {
        1: [1.0, 0.6, 0.6],   # Light Red
        2: [0.6, 1.0, 0.6],   # Light Green
        3: [0.6, 0.6, 1.0],   # Light Blue
        4: [1.0, 1.0, 0.6],   # Light Yellow
        5: [1.0, 0.6, 1.0],   # Light Magenta
        6: [0.6, 1.0, 1.0],   # Light Cyan
        7: [0.8, 0.7, 1.0],   # Light Purple
        8: [1.0, 0.8, 0.6]    # Light Orange
    }
    
    # Create an RGB mask initialized with zeros
    mask_rgb = np.zeros_like(overlay)

    
    # Assign colors based on binary_mask values
    for value, color in colors.items():
        mask_rgb[binary_mask == value] = color
    
    # Define transparency level
    alpha = 0.4  # Transparency level (0-1)
    
    # Blend grayscale image with the colored mask
    blended = overlay * (1 - alpha) + mask_rgb * alpha
    blended = (blended*255).astype(np.uint8)
    image = Image.fromarray(blended)
    index = len(os.listdir('Predictions'))+1
    image.save(f'Predictions/output_{index}.jpg', quality=100)

In [12]:
import numpy as np

def normalize_gray_image(gray_image):
    min_val = np.min(gray_image)
    max_val = np.max(gray_image)
    
    # Avoid division by zero
    if max_val == min_val:
        return np.zeros_like(gray_image, dtype=np.uint8)

    normalized_image = (gray_image - min_val) / (max_val - min_val) * 255
    return normalized_image.astype(np.uint8)

# Example usage:
gray_image = np.array([-0.4436477, -0.4436453, -0.4436401, 3.9032059, 3.9032698, 3.9033058])
normalized_image = normalize_gray_image(gray_image)

print(normalized_image)


[  0   0   0 254 254 255]


In [13]:
for batch_idx, (inputs, targets) in enumerate(train_loader):
    # inputs, targets = inputs.to(device), targets.to(device)
    inputs = np.array(inputs)
    targets = np.array(targets)
    for batch in range(inputs.shape[0]):
        for sli in range(inputs.shape[2]):
            
            gray_image = inputs[batch, 0, sli, :, :]
            binary_mask = targets[batch, 0, sli, :, :]
            gray_image = normalize_gray_image(gray_image)
            print('gray_image:-',np.unique(gray_image))
            print('binary_mask:-',np.unique(binary_mask))
            if len(np.unique(binary_mask))>1:
                # print('binary_mask:-',np.unique(binary_mask))
                save_prediction(gray_image, binary_mask)

C:\Users\Rishabh\AppData\Local\Temp\ipykernel_21340\3510380900.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  inputs = np.array(inputs)
C:\Users\Rishabh\AppData\Local\Temp\ipykernel_21340\3510380900.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  targets = np.array(targets)


gray_image:- [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 2

KeyboardInterrupt: 